In [1]:
from ultralytics import YOLO
from mmocr.apis import MMOCRInferencer
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import torch
import cv2
import os
import csv
from os import walk
import pandas as pd
import re
import numpy as np

2023-10-14 16:57:22,225 - modelscope - INFO - PyTorch version 2.0.1+cu117 Found.
2023-10-14 16:57:22,230 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2023-10-14 16:57:22,231 - modelscope - INFO - Loading ast index from C:\Users\Олег\.cache\modelscope\ast_indexer
2023-10-14 16:57:22,431 - modelscope - INFO - Loading done! Current index file version is 1.9.2, with md5 d900a8624d792d555ed3cef91c01c35b and a total number of 941 components indexed


In [3]:
'''
    detection_mode: ./models/custom_yolov8pt_25_orig.pt
    rec_model: damo/cv_convnextTiny_ocr-recognition-general_damo
    angle_rec_model: Aster
'''

DETECTION_SAVE_PATH = './yolo_detections/results/crops/number/'
MODEL_RESULT_PATH = './model_result/results.csv'

BIN_TYPES = {
    'ADAPTIVE_THRESH_GAUSSIAN_C': cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    'ADAPTIVE_THRESH_MEAN_C': cv2.ADAPTIVE_THRESH_MEAN_C,
    'THRESH_OTSU': cv2.THRESH_OTSU
}

# def get_center(bbox):
#     return np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])
#
# def is_border_num(result, image_path):
#         for i in range(len(result)):
#             boxes = result[i].boxes.cpu().numpy()
#             for box in boxes:
#                 center = get_center(box.xyxy[0])
#
#
#                 img = cv2.imread(image_path)
#
#                 if center[0] < (img.shape[1] / 10) or center[0] > (img.shape[1] / 10) * 9:
#                     return True
#             return False


class NumberOcrModel:
    def __init__(self, detection_model, rec_model, angle_rec_model):
        self.detection_model = YOLO(detection_model)
        self.rec_model = pipeline(Tasks.ocr_recognition, model=rec_model)
        self.angle_rec_model = MMOCRInferencer(rec=angle_rec_model)

        self.detection_result = None
        self.img_path = None
        self.image_name = None
        self.bin_type = None
        self.rec_result = None

        self.prepare_model()

    def prepare_model(self):
        if torch.cuda.is_available():
            self.detection_model.to('cuda')

    def preprocess(self, image_path, image_name, bin_prep):
        detection_result = self.detection_model.predict(image_path, save = True, save_crop=True, project='yolo_detections', name='results', verbose=False)

        if detection_result and bin_prep:
            img = cv2.imread(DETECTION_SAVE_PATH + image_name)
            blur_img = cv2.GaussianBlur(img,(1,1),0)
            bin_img = cv2.adaptiveThreshold(blur_img, 255, BIN_TYPES[bin_prep], cv2.THRESH_BINARY_INV, 29, -4)
            cv2.imwrite(DETECTION_SAVE_PATH + image_name, bin_img)
        return detection_result


    def recognize(self, image_name, image_path, detected_data):
        if not detected_data[0] or len(detected_data[0].cpu().numpy()) == 0:
            # return [{
            #     'filename': image_name,
            #     'type': 0,
            #     'number': 0,
            #     'is_correct': 0,
            #     'model': 'Recognition_model',
            #     'is_correct_rec': False
            # },
            # {
            #     'filename': image_name,
            #     'type': 0,
            #     'number': 0,
            #     'is_correct': 0,
            #     'model': 'Angle_recognition_model',
            #     'is_correct_rec': False
            # }]

            return [
                {
                    'filename': image_name,
                    'type': 0,
                    'number': 0,
                    'is_correct': 0,
                    'model': '-',
                    'is_correct_rec': False
                }]

        all_dirs = os.listdir('./yolo_detections')
        max_length = len(max(all_dirs, key=len))
        data_dir = sorted([x for x in all_dirs if len(x) == max_length])[-1]

        crop_img_path = f'./yolo_detections/{data_dir}/crops/number/' + image_name

        # if is_border_num(detected_data, image_path):
        #     result = self.angle_rec_model(crop_img_path)
        #     num =  re.sub(r'[^0-9]', '', result['predictions'][0]['rec_texts'][0])
        #     model = 'Angle_recognition_model'
        # else:
        #     result = self.rec_model(crop_img_path)
        #     num = re.sub(r'[^0-9]', '', result['text'][0])
        #     model = 'Recognition_model'


        result_1 = self.angle_rec_model(crop_img_path)
        result_2 = self.rec_model(crop_img_path)

        num_1 = re.sub(r'[^0-9]', '', result_1['predictions'][0]['rec_texts'][0]) # aster
        num_2 = re.sub(r'[^0-9]', '', result_2['text'][0]) # model scope


        if not num_2:
            num_sub = num_1
        elif num_1:
            if len(num_2) >= 8:
                num_sub = num_2[:8]
            if len(num_2) < 8 and len(num_1) >= 8:
                num_sub = num_1[:8]
            if len(num_2) < 8 and len(num_1) < 8:
                num_sub = num_2


        # rec_result = self.rec_model(crop_img_path)
        # angle_rec_result = self.angle_rec_model(crop_img_path)

        # num_1 = re.sub(r'[^0-9]', '', rec_result['text'][0])
        # num_2 =  re.sub(r'[^0-9]', '', angle_rec_result['predictions'][0]['rec_texts'][0])


        # result = [
        #     {
        #         'filename': image_name,
        #         'type': (0, 1)[len(num_1) > 0],
        #         'number': num_1,
        #         'is_correct': is_valid(num_1),
        #         'model': 'Recognition_model',
        #         'is_correct_rec': num_1 == image_name[:-4]
        #     },
        #     {
        #         'filename': image_name,
        #         'type': (0, 1)[len(num_2) > 0],
        #         'number': num_2,
        #         'is_correct': is_valid(num_2),
        #         'model': 'Angle_recognition_model',
        #         'is_correct_rec': num_2 == image_name[:-4]
        #     }
        # ]

        result = [{
                'filename': image_name,
                'type': int(num_sub != None),
                'number': (0, num_sub)[num_sub != None],
                'is_correct': is_valid(num_sub),
                # 'model': model,
                # 'is_correct_rec': num_sub == image_name[:-4]
        }]

        return result

    def predict(self, img_path, bin_prep = None):
        image_name = os.path.basename(os.path.normpath(img_path))
        detected_data = self.preprocess(img_path, image_name, bin_prep)
        return self.recognize(image_name, img_path, detected_data)



In [4]:
model = NumberOcrModel(
    detection_model='./models/best(6).pt',  # best(6).pt ./models/custom_yolov8pt_25_orig.pt
    rec_model='damo/cv_convnextTiny_ocr-recognition-general_damo',
    angle_rec_model='Aster'
)

2023-10-14 17:10:34,718 - modelscope - INFO - Model revision not specified, use revision: v2.3.0
2023-10-14 17:10:35,925 - modelscope - INFO - initiate model from C:\Users\Олег\.cache\modelscope\hub\damo\cv_convnextTiny_ocr-recognition-general_damo
2023-10-14 17:10:35,926 - modelscope - INFO - initiate model from location C:\Users\Олег\.cache\modelscope\hub\damo\cv_convnextTiny_ocr-recognition-general_damo.
2023-10-14 17:10:35,930 - modelscope - INFO - initialize model from C:\Users\Олег\.cache\modelscope\hub\damo\cv_convnextTiny_ocr-recognition-general_damo
2023-10-14 17:10:36,259 - modelscope - INFO - loading model from dir C:\Users\Олег\.cache\modelscope\hub\damo\cv_convnextTiny_ocr-recognition-general_damo
2023-10-14 17:10:36,440 - modelscope - INFO - loading model done


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/aster/aster_resnet45_6e_st_mj/aster_resnet45_6e_st_mj-cc56eca4.pth
10/14 17:10:43 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.


C:\temp\Anaconda\lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [27]:
test_res = model.predict('./data/NONE11.jpg')
print(test_res)

Results saved to yolo_detections\results734


[{'filename': 'NONE11.jpg', 'type': 0, 'number': 0, 'is_correct': 0, 'model': 'Recognition_model', 'is_correct_rec': False}, {'filename': 'NONE11.jpg', 'type': 0, 'number': 0, 'is_correct': 0, 'model': 'Angle_recognition_model', 'is_correct_rec': False}]


[{'filename': 'NONE11.jpg',
  'type': 0,
  'number': 0,
  'is_correct': 0,
  'model': 'Recognition_model',
  'is_correct_rec': False},
 {'filename': 'NONE11.jpg',
  'type': 0,
  'number': 0,
  'is_correct': 0,
  'model': 'Angle_recognition_model',
  'is_correct_rec': False}]

In [14]:
def is_valid(result):
    if not result:
        return 0

    cont_sum = 0
    control_num = -1

    if len(result) == 8:
        control_num = int(result[-1:])
        cont_sum = 0
        for i in range(7):
            num = int(result[i]) * (2, 1)[i % 2 == 1]
            if num >= 10:
                cont_sum += sum(list(map(int, set(str(num)))))
            else:
                cont_sum += num
    return int((cont_sum % 10 == 0 and control_num == 0) or (10 - cont_sum % 10) == control_num)


def to_csv(results):
    with open(MODEL_RESULT_PATH, 'w', encoding='UTF8') as f:
        fields = ('filename', 'type', 'number', 'is_correct')  # 'model', 'is_correct_rec'
        writer = csv.DictWriter(f, fieldnames=fields, lineterminator = '\n')
        writer.writeheader()
        for res in results:
            writer.writerow(res[0])

In [15]:
DATA_PATH = './test_images/'

images = []

for (dirpath, dirnames, filenames) in walk(DATA_PATH):
    images.extend(filenames)

results = []

for img in images:
    res = model.predict(DATA_PATH + img)
    if res:
        results.append(res)

to_csv(results)

file = pd.read_csv(MODEL_RESULT_PATH)
print(file)

Results saved to yolo_detections\results862


Output()

Results saved to yolo_detections\results863


Output()

Results saved to yolo_detections\results864


...

Results saved to yolo_detections\results869


Output()

Results saved to yolo_detections\results870


Output()

       filename  type    number  is_correct              model  is_correct_rec
0  28005312.jpg     1  28005312           1  Recognition_model            True
1  28008332.jpg     1  28008332           1  Recognition_model            True
2  28025021.jpg     1  28025021           1  Recognition_model            True
3  29025210.jpg     1  29025210           1  Recognition_model            True
4  29029972.jpg     1  29029972           1  Recognition_model            True
5  29051091.jpg     1  29051091           1  Recognition_model            True
6  42026633.jpg     1  42026633           1  Recognition_model            True
7  42026781.jpg     1  42026781           1  Recognition_model            True
8  42026872.jpg     1  42026872           1  Recognition_model            True


In [ ]:
import re
line = re.sub(r'[^0-9]', '', '1,2')

In [ ]:
line

In [ ]:
a = 'sdfsdf.jpg'
a[:-4]

In [8]:
file = pd.read_csv(MODEL_RESULT_PATH)

In [16]:
rec = file[file['model'] == 'Recognition_model']

In [17]:
rec[rec['is_correct_rec']]

,filename,type,number,is_correct,model,is_correct_rec
0,24252710.jpg,1,24252710.0,0,Recognition_model,True
2,24295479.jpg,1,24295479.0,1,Recognition_model,True
4,24353013.jpg,1,24353013.0,1,Recognition_model,True
6,24424020.jpg,1,24424020.0,0,Recognition_model,True
8,24432064.jpg,1,24432064.0,1,Recognition_model,True
...,...,...,...,...,...,...
1446,98091358.jpg,1,98091358.0,1,Recognition_model,True
1448,98101173.jpg,1,98101173.0,1,Recognition_model,True
1450,98104730.jpg,1,98104730.0,0,Recognition_model,True
1454,98121817.jpg,1,98121817.0,1,Recognition_model,True
